In [0]:
%run /Workspace/Users/ajkayode@outlook.com/ehr-project/ehr-project-bundle/envsetup/Configs

In [0]:
class ProfileBronzeLoad:
  def __init__(self):
      pass
  def make_source_dir(self):
    src_dir = (
        spark.sql(
            f"""select date_add(last_load_date, 1) as source_dir 
            from {ats_configs.jobs_metadate_table_name} 
            where job_name = '{ats_configs.profile_bronze_job_name}'
            order by last_load_date desc
            """).first()
                .asDict()["source_dir"]
                .strftime("%Y-%m-%d")
    )
    return src_dir
    
  def update_metadata(self, ingestion_date):
    print(f"Updating metadata for {ats_configs.profile_bronze_job_name}")
    spark.sql(
        f"""
        insert into {ats_configs.jobs_metadate_table_name}
        values('{ats_configs.profile_bronze_job_name}', 
            '{ingestion_date}', 
            current_timestamp(), 
            'Job Execution Completed.' )""")
    print(f"Updated metadata for {ats_configs.profile_bronze_job_name}")
  
  def load_bronze(self):
    print(f"Loading bronze data for {ats_configs.profile_bronze_job_name}")
    src_dir = self.make_source_dir()
    path = f"/Volumes/{ats_configs.catalog}/{ats_configs.db}/{ats_configs.profile_landing_zone}/{src_dir}"
    df = (
        spark.read.format("binaryFile")
        .option("recursiveFileLookup", "true")
        .option("maxFilesPerTrigger", 10)
        .option("pathGlobFilter", "*.pdf")
        .load(f"{path}")
    )
    df.write.mode("append").saveAsTable(ats_configs.profile_bronze_table_name)
    self.update_metadata(src_dir)
    print(f"Bronze data load completed for {ats_configs.profile_bronze_table_name}")

  def assert_count(self, table_name, expected_count):
      print(f"Asserting count for {table_name}")
      actual_count = spark.read.table(f"{ats_configs.catalog}.{ats_configs.db}.{table_name}").count()
      assert (actual_count == expected_count), f"Expected {expected_count} records in {table_name}, found {actual_count}."
      print(f"Asserted count for {table_name}")

  def validate(self, iter):
    import time
    start = time.time()
    print("\nValidating profile load to bronze...")
    for file in iter:
      self.assert_count(ats_configs.profile_bronze_table_name, 5 if iter == 1 else 10)
    end = time.time()
    print(f"\nValidated profile bronze load in {end - start} seconds.")

      

In [0]:
ProfileBronze = ProfileBronzeLoad()
ProfileBronze.load_bronze()